<a href="https://www.kaggle.com/code/sagorkumarmitra/text-classification?scriptVersionId=190497442" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('/kaggle/input/smsspamcollection/smsspamcollection.tsv', sep='\t')
df.head()

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2


In [3]:
df.isnull().sum()

label      0
message    0
length     0
punct      0
dtype: int64

In [4]:
len(df)

5572

In [5]:
df['label'].unique()

array(['ham', 'spam'], dtype=object)

In [6]:
from sklearn.model_selection import train_test_split

X = df['message']  # this time we want to look at the text
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape

(3733, 7082)

In [8]:
X_train_counts

<3733x7082 sparse matrix of type '<class 'numpy.int64'>'
	with 49992 stored elements in Compressed Sparse Row format>

In [9]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()

X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

X_train_tfidf.shape

(3733, 7082)

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

X_train_tfidf = vectorizer.fit_transform(X_train)

In [11]:
from sklearn.svm import LinearSVC

clf = LinearSVC()

clf.fit(X_train_tfidf,y_train)

LinearSVC()

In [12]:
from sklearn.pipeline import Pipeline

text_clf = Pipeline([('tfidf',TfidfVectorizer()), ('clf',LinearSVC())])

text_clf.fit(X_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [13]:
predictions = text_clf.predict(X_test)

In [14]:
X_test

3245    Squeeeeeze!! This is christmas hug.. If u lik ...
944     And also I've sorta blown him off a couple tim...
1044    Mmm thats better now i got a roast down me! i...
2484        Mm have some kanji dont eat anything heavy ok
812     So there's a ring that comes with the guys cos...
                              ...                        
4944    Check mail.i have mailed varma and kept copy t...
3313    I know you are serving. I mean what are you do...
3652         Want to send me a virtual hug?... I need one
14                    I HAVE A DATE ON SUNDAY WITH WILL!!
4758    hey, looks like I was wrong and one of the kap...
Name: message, Length: 1839, dtype: object

In [15]:
y_test

3245    ham
944     ham
1044    ham
2484    ham
812     ham
       ... 
4944    ham
3313    ham
3652    ham
14      ham
4758    ham
Name: label, Length: 1839, dtype: object

In [16]:
predictions

array(['ham', 'ham', 'ham', ..., 'ham', 'ham', 'ham'], dtype=object)

In [17]:
from sklearn.metrics import confusion_matrix,classification_report

print(confusion_matrix(y_test,predictions))

print(classification_report(y_test,predictions))

[[1586    7]
 [  12  234]]
              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1593
        spam       0.97      0.95      0.96       246

    accuracy                           0.99      1839
   macro avg       0.98      0.97      0.98      1839
weighted avg       0.99      0.99      0.99      1839



In [18]:
from sklearn import metrics

metrics.accuracy_score(y_test,predictions)

0.989668297988037

In [19]:
text_clf.predict(["Hi how are you doing?"])

array(['ham'], dtype=object)

## Movie Review Text Classification

In [20]:
import numpy as np
import pandas as pd

df = pd.read_csv('/kaggle/input/moviereviews/moviereviews.tsv', sep='\t')
df.head()

,label,review
0,neg,how do films like mouse hunt get into theatres...
1,neg,some talented actresses are blessed with a dem...
2,pos,this has been an extraordinary year for austra...
3,pos,according to hollywood movies made in last few...
4,neg,my first press screening of 1998 and already i...


In [21]:
len(df)

2000

In [22]:
print(df['review'][0])

how do films like mouse hunt get into theatres ? 
isn't there a law or something ? 
this diabolical load of claptrap from steven speilberg's dreamworks studio is hollywood family fare at its deadly worst . 
mouse hunt takes the bare threads of a plot and tries to prop it up with overacting and flat-out stupid slapstick that makes comedies like jingle all the way look decent by comparison . 
writer adam rifkin and director gore verbinski are the names chiefly responsible for this swill . 
the plot , for what its worth , concerns two brothers ( nathan lane and an appalling lee evens ) who inherit a poorly run string factory and a seemingly worthless house from their eccentric father . 
deciding to check out the long-abandoned house , they soon learn that it's worth a fortune and set about selling it in auction to the highest bidder . 
but battling them at every turn is a very smart mouse , happy with his run-down little abode and wanting it to stay that way . 
the story alternates betwee

In [23]:
df.isnull().sum()

label      0
review    35
dtype: int64

In [24]:
df.dropna(inplace=True)

In [25]:
df.isnull().sum()

label     0
review    0
dtype: int64

In [26]:
blanks = []

for i,lb,rv in df.itertuples():
    if rv.isspace():
        blanks.append(i)

In [27]:
df.drop(blanks,inplace=True)

In [28]:
len(df)

1938

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix,classification_report

X = df['review']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

text_clf = Pipeline([('tfidf', TfidfVectorizer()),('clf',LinearSVC())])

text_clf.fit(X_train,y_train)

predictions = text_clf.predict(X_test)

print(confusion_matrix(y_test,predictions))

print(classification_report(y_test,predictions))

metrics.accuracy_score(y_test,predictions)

[[242  50]
 [ 32 258]]
              precision    recall  f1-score   support

         neg       0.88      0.83      0.86       292
         pos       0.84      0.89      0.86       290

    accuracy                           0.86       582
   macro avg       0.86      0.86      0.86       582
weighted avg       0.86      0.86      0.86       582



0.8591065292096219

## Another Movie Review Text Classification

In [30]:
import numpy as np
import pandas as pd

df = pd.read_csv('/kaggle/input/moviereviews2/moviereviews2.tsv', sep='\t')
df.isnull().sum()

df.dropna(inplace=True)

df.head()

,label,review
0,pos,I loved this movie and will watch it again. Or...
1,pos,"A warm, touching movie that has a fantasy-like..."
2,pos,I was not expecting the powerful filmmaking ex...
3,neg,"This so-called ""documentary"" tries to tell tha..."
4,pos,This show has been my escape from reality for ...


In [31]:
blanks = []

for i,lb,rv in df.itertuples():
    if rv.isspace():
        blanks.append(i)
        
df.drop(blanks,inplace=True)

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix,classification_report

X = df['review']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

text_clf = Pipeline([('tfidf', TfidfVectorizer()),('clf',LinearSVC())])

text_clf.fit(X_train,y_train)

predictions = text_clf.predict(X_test)

print(confusion_matrix(y_test,predictions))

print(classification_report(y_test,predictions))

metrics.accuracy_score(y_test,predictions)

[[832  75]
 [ 65 822]]
              precision    recall  f1-score   support

         neg       0.93      0.92      0.92       907
         pos       0.92      0.93      0.92       887

    accuracy                           0.92      1794
   macro avg       0.92      0.92      0.92      1794
weighted avg       0.92      0.92      0.92      1794



0.9219620958751393